1. Introduction et Objectifs
Après avoir exploré le Content-Based Filtering (KNN) et le Machine Learning Supervisé (Random Forest), nous abordons ici la troisième famille de recommandation : le Collaborative Filtering.

L'objectif de ce notebook : Implémenter l'algorithme SVD (Singular Value Decomposition).

Contrairement aux modèles précédents, le SVD ne regarde pas le synopsis ou les genres.

Il se base uniquement sur la matrice d'interactions : "Qui a aimé quoi".

Il permet de découvrir des relations cachées (Latent Factors) entre les utilisateurs et les animes.

Note : Ce notebook utilise la bibliothèque scikit-surprise. Assure-toi de l'avoir installée : pip install scikit-surprise.

2. Imports et Configuration

In [62]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from datetime import datetime
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix # Pour gérer la mémoire efficacement
import pickle
# Configuration
pd.set_option('display.max_columns', None)
sns.set_theme(style="whitegrid")

3. Chargement des Interactions (User Ratings)
Le SVD a besoin d'un fichier où l'on voit quel utilisateur (user_id) a donné quelle note (rating) à quel anime (anime_id).

In [63]:
# --- CELLULE 3 : Chargement depuis RAW ---
DATA_PATH = "../data/raw/favs.csv"

# On charge le début du fichier pour vérifier le format sans tout bloquer
df_check = pd.read_csv(DATA_PATH, nrows=5)
print("🔍 Colonnes détectées dans le fichier :", df_check.columns.tolist())

# Chargement complet (on force les types pour gagner de la RAM)
# Si ton fichier est ENORME, ajoute nrows=1000000 pour tester
df_favs = pd.read_csv(DATA_PATH)

# On nettoie les noms de colonnes (enlève les espaces et met en minuscule)
df_favs.columns = [c.strip().lower() for c in df_favs.columns]

print(f"✅ Fichier chargé : {df_favs.shape}")
df_favs.head()

🔍 Colonnes détectées dans le fichier : ['username', 'fav_type', 'id']
✅ Fichier chargé : (4178747, 3)


,username,fav_type,id
0,ishikawas,anime,45649
1,ishikawas,anime,38680
2,ishikawas,anime,795
3,ishikawas,anime,37510
4,ishikawas,anime,820


4. Analyse de la Matrice (Sparsity)
Étape indispensable en Master pour justifier l'usage du SVD.

In [64]:
# --- CELLULE 4 : Analyse robuste et détection ---

# 1. On affiche les colonnes pour ne plus avancer à l'aveugle
cols = df_favs.columns.tolist()
print(f"🔍 Colonnes présentes dans le fichier : {cols}")

# 2. Détection intelligente
try:
    # On cherche l'utilisateur (user, user_id, uid...)
    col_user = [c for c in cols if 'user' in c or 'uid' in c][0]
    
    # On cherche l'anime : ce qui contient 'anime' OU ce qui contient 'id' 
    # mais qui n'est PAS la colonne utilisateur
    potential_anime_cols = [c for c in cols if ('anime' in c or 'id' in c) and c != col_user]
    
    if not potential_anime_cols:
        # Si on ne trouve rien, on prend la deuxième colonne par défaut
        col_anime = cols[1]
    else:
        col_anime = potential_anime_cols[0]

    print(f"📌 Identification réussie : Utilisateur = '{col_user}', Anime = '{col_anime}'")

except IndexError:
    print("❌ Échec de la détection automatique.")
    # Valeurs de secours (on prend les deux premières colonnes)
    col_user, col_anime = cols[0], cols[1]
    print(f"⚠️ Utilisation par défaut des colonnes : '{col_user}' et '{col_anime}'")

# 3. Calcul des statistiques
n_users = df_favs[col_user].nunique()
n_animes = df_favs[col_anime].nunique()
sparsity = 1.0 - (len(df_favs) / (n_users * n_animes))

print(f"\n📊 Statistiques de la Matrice :")
print(f"   - Utilisateurs uniques : {n_users}")
print(f"   - Animes uniques      : {n_animes}")
print(f"   - Sparsity            : {sparsity:.4%}")

🔍 Colonnes présentes dans le fichier : ['username', 'fav_type', 'id']
📌 Identification réussie : Utilisateur = 'username', Anime = 'id'

📊 Statistiques de la Matrice :
   - Utilisateurs uniques : 246095
   - Animes uniques      : 51585
   - Sparsity            : 99.9671%


5. Prétraitement et Pivot Table
On transforme la liste en matrice Animes x Utilisateurs.

In [65]:
# --- CELLULE 5 : Filtrage et Matrice Pondérée (TF-IDF) ---
from sklearn.feature_extraction.text import TfidfTransformer

# 1. On garde ton filtrage de base
min_user_favs = 5
min_anime_favs = 150
user_counts = df_favs[col_user].value_counts()
df_filtered = df_favs[df_favs[col_user].isin(user_counts[user_counts >= min_user_favs].index)].copy()
anime_counts = df_filtered[col_anime].value_counts()
df_filtered = df_filtered[df_filtered[col_anime].isin(anime_counts[anime_counts >= min_anime_favs].index)].copy()

# 2. Encodage
df_filtered['user_cat'] = df_filtered[col_user].astype('category')
df_filtered['anime_cat'] = df_filtered[col_anime].astype('category')

# 3. Création de la matrice binaire
user_anime_sparse = csr_matrix(
    (np.ones(len(df_filtered)), (df_filtered['anime_cat'].cat.codes, df_filtered['user_cat'].cat.codes))
)

# 4. PONDÉRATION TF-IDF (Nouveau : crucial pour la pertinence)
# Cela réduit l'importance des utilisateurs "bourrins" et booste les "sélectifs"
tfidf = TfidfTransformer()
user_anime_weighted = tfidf.fit_transform(user_anime_sparse)

mapped_anime_ids = df_filtered['anime_cat'].cat.categories
print(f"✅ Matrice pondérée créée : {user_anime_weighted.shape}")

✅ Matrice pondérée créée : (3064, 209291)


6. Entraînement du Modèle SVD

In [66]:
# --- CELLULE 6 : SVD Haute Résolution ---
n_components = 1000
svd = TruncatedSVD(n_components=n_components, random_state=42)

# On entraîne sur la matrice pondérée (weighted)
latent_matrix = svd.fit_transform(user_anime_weighted)

explained_variance = svd.explained_variance_ratio_.sum()
print(f"🧠 Modèle entraîné ({n_components} composantes).")
print(f"📈 Variance expliquée : {explained_variance:.2%} (Cible > 40%)")

🧠 Modèle entraîné (1000 composantes).
📈 Variance expliquée : 43.66% (Cible > 40%)


7. Calcul de la Similarité Cosinus
On calcule la ressemblance entre les animes dans l'espace réduit.

In [67]:
# Calcul de la similarité cosinus dans l'espace réduit (Latent Space)
# On convertit en float32 pour économiser 50% de la RAM
item_similarity = cosine_similarity(latent_matrix).astype(np.float32)

df_similarity = pd.DataFrame(
    item_similarity, 
    index=mapped_anime_ids, 
    columns=mapped_anime_ids
)
del item_similarity 
print("✅ Matrice de similarité prête.")

print(f"✅ Matrice de similarité collaborative générée : {df_similarity.shape}")
print(f"📦 Taille en mémoire : {df_similarity.memory_usage().sum() / 1e6:.2f} MB")

✅ Matrice de similarité prête.
✅ Matrice de similarité collaborative générée : (3064, 3064)
📦 Taille en mémoire : 37.64 MB


8. Sauvegarde des Artefacts

In [68]:
run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir = f"../runs/svd_collab_{run_id}/artifacts/"
os.makedirs(save_dir, exist_ok=True)

# On sauvegarde la matrice de similarité pour l'utiliser dans le modèle hybride
SVD_EXPORT_PATH = "../data/processed/svd_similarity_matrix.pkl"

# Utilisation de pickle pour conserver la structure DataFrame et les types numériques
df_similarity.to_pickle(SVD_EXPORT_PATH)

print(f"✅ Succès : La matrice SVD a été exportée vers {SVD_EXPORT_PATH}")

joblib.dump(svd, f"{save_dir}svd_model.joblib")
joblib.dump(df_similarity, f"{save_dir}similarity_matrix.joblib")
print(f"✅ Artefacts sauvegardés dans : {save_dir}")

✅ Succès : La matrice SVD a été exportée vers ../data/processed/svd_similarity_matrix.pkl
✅ Artefacts sauvegardés dans : ../runs/svd_collab_20251228_002038/artifacts/


9. Test du Modèle (Inférence Collaborative)

In [69]:
def get_collaborative_recommendations(anime_title, n_recs=10):
    df_master = pd.read_csv("../data/processed/anime_master_clean.csv")
    id_col = 'mal_id' # Ton champ pivot
    
    matches = df_master[df_master['title'].str.contains(anime_title, case=False, na=False)]
    if matches.empty: return "Animé non trouvé."
    
    target_data = matches.sort_values(by='members', ascending=False).iloc[0]
    target_id = target_data[id_col]
    
    # 1. Obtenir les scores SVD bruts
    if target_id not in df_similarity.index:
        return f"⚠️ Pas assez de données pour {target_data['title']}."
    
    similar_scores = df_similarity.loc[target_id]
    
    # 2. RÉPARATION : Pondération par la popularité (Log-Scaling)
    # On récupère les candidats
    candidate_ids = similar_scores.index
    recs = df_master[df_master[id_col].isin(candidate_ids)].copy()
    
    # On récupère le score SVD
    recs['svd_pure'] = recs[id_col].map(similar_scores)
    
    # --- LA FORMULE DE RÉPARATION ---
    # On multiplie le score SVD par le log de la popularité pour favoriser les titres solides
    # tout en laissant une petite chance aux niches cohérentes.
    recs['final_score'] = recs['svd_pure'] * np.log10(recs['members'])
    
    # 3. Nettoyage final (on enlève la cible elle-même)
    result = recs[recs[id_col] != target_id].sort_values(by='final_score', ascending=False)
    
    print(f"✨ Résultats corrigés pour : {target_data['title']}")
    return result.head(n_recs)[['title', 'genres_list', 'score', 'members', 'final_score']]

In [70]:

def test_repaired_model(anime_name, n_recs=10):
    # 1. Chargement de ta source de vérité
    df_master = pd.read_csv("../data/processed/anime_master_clean.csv")
    
    # 2. On trouve l'ID mal_id de ton animé
    matches = df_master[df_master['title'].str.contains(anime_name, case=False, na=False)]
    if matches.empty: return print(f"❌ '{anime_name}' non trouvé.")
    
    target = matches.sort_values(by='members', ascending=False).iloc[0]
    t_id = target['mal_id']
    
    if t_id not in df_similarity.index:
        return print(f"⚠️ Pas assez de données pour '{target['title']}'")

    # 3. Calcul du score REPARÉ
    # On récupère tous les scores de similarité pour cet animé
    sim_scores = df_similarity.loc[t_id]
    
    # On crée un DataFrame de travail
    df_recs = df_master[df_master['mal_id'].isin(sim_scores.index)].copy()
    df_recs['svd_pure'] = df_recs['mal_id'].map(sim_scores)
    
    # --- LA FORMULE DE RÉPARATION ---
    # On multiplie la similarité par le log de la popularité (members)
    # Cela donne un coup de pouce aux animés que beaucoup de gens ont validé
    df_recs['final_score'] = df_recs['svd_pure'] * np.log10(df_recs['members'])
    
    # 4. On prépare l'affichage
    # On enlève l'animé lui-même
    df_recs = df_recs[df_recs['mal_id'] != t_id]
    
    # Tri par le nouveau score
    result = df_recs.sort_values(by='final_score', ascending=False).head(n_recs)
    
    print(f"🚀 TEST DE RÉCOMPENSE POUR : {target['title']}")
    print(f"📊 Basé sur {len(df_favs)} interactions utilisateurs + Ton Master Clean")
    print("-" * 80)
    
    return result[['title', 'genres_list', 'score', 'members', 'svd_pure', 'final_score']]

# --- LANCEMENT DU TEST ---
test_repaired_model("Naruto")

🚀 TEST DE RÉCOMPENSE POUR : Naruto
📊 Basé sur 4178747 interactions utilisateurs + Ton Master Clean
--------------------------------------------------------------------------------


,title,genres_list,score,members,svd_pure,final_score
16826,Naruto: Shippuuden,"['Action', 'Adventure', 'Fantasy']",8.28,2668197,0.759473,4.880538
9659,Hungry Heart: Wild Striker,['Sports'],7.54,26418,0.659890,2.917968
12209,Kidou Senshi Zeta Gundam,"['Action', 'Drama', 'Romance', 'Sci-Fi']",7.89,84577,0.486050,2.394890
22002,Shingeki no Kyojin,"['Action', 'Award Winning', 'Drama', 'Suspense']",8.56,4230312,0.341280,2.261449
15610,Midnight Eye: Gokuu,"['Action', 'Mystery', 'Sci-Fi']",6.47,15705,0.490729,2.059116
2793,Boruto: Naruto Next Generations,"['Action', 'Adventure', 'Fantasy']",5.99,913270,0.336461,2.005508
17912,One Piece,"['Action', 'Adventure', 'Fantasy']",8.73,2581334,0.312016,2.000598
2493,Bleach,"['Action', 'Adventure', 'Supernatural']",7.98,2134926,0.311201,1.969710
4780,Death Note,"['Supernatural', 'Suspense']",8.62,4170767,0.283397,1.876150
24866,Tokimeki Memorial: Only Love,"['Comedy', 'Romance']",7.01,39023,0.358832,1.647513
